<div align="center">

  <a href="https://ultralytics.com/yolov5" target="_blank">
    <img width="1024", src="https://github.com/ultralytics/assets/raw/master/yolov5/v62/splash_notebook.png"></a>


<br>
  <a href="https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
<br>

This <a href="https://github.com/ultralytics/yolov5">YOLOv5</a> 🚀 notebook by ROAR Lab presents simple train, validate and predict examples to help start your AI adventure.<br>Credit of this notebook goes to <a href="https://ultralytics.com/">Ultralytics</a> and <a href="https://roboflow.com/">Roboflow</a> as this notebook used both sample as reference.

</div>

## Things to Edit ##
- To change the background of the YOLOv5 logo as well as the link buttons to STEAMxD
- I need to include steps 

# Setup

Mount GDrive, Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [ ]:
# check that you're running GPU instance
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
from IPython.display import Image, clear_output  # to display images
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.2-56-g30e674b Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 37.4/166.8 GB disk)


# 1. Image Augmentation

In this example we will do a single augmentation (mosaic) and train the model for a single epoch to showcase the available augmentations while training.

<!---
To edit the instructions to include Augmentation discussions for ease of understanding


`detect.py` runs YOLOv5 inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image 
                          vid.mp4  # video
                          path/  # directory
                          'path/*.jpg'  # glob
                          'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                          'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```
-->

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Image Augmentation Parameters

- To insert images of the examples of each augmentation
- The table below is temporary need to edit the necessary images in

| Class     | Points |
| :-:       | :-:    |
| Humans    | 5      |
| Pets      | 3      |
| Valuables | 1      |

`l_gamma`: 0.0  # focal loss gamma (efficientDet default gamma=1.5)\
`hsv_h`: 0.0  # image HSV-Hue augmentation (fraction)\
`hsv_s`: 0.0  # image HSV-Saturation augmentation (fraction)\
`hsv_v`: 0.0  # image HSV-Value augmentation (fraction)\
`degrees`: 0.0  # image rotation (+/- deg)\
`translate`: 0.0  # image translation (+/- fraction)\
`scale`: 0.0  # image scale (+/- gain)\
`shear`: 0.0  # image shear (+/- deg)\
`perspective`: 0.0  # image perspective (+/- fraction), range 0-0.001\
`flipud`: 0.0  # image flip up-down (probability)\
`fliplr`: 0.0  # image flip left-right (probability)\
`mosaic`: 1.0  # image mosaic (probability)\
`mixup`: 0.0  # image mixup (probability)

In [ ]:
# how to rewrite the hyps yaml
%%writetemplate /content/yolov5/data/hyps/hyp.scratch-low.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for low-augmentation COCO training from scratch
# python train.py --batch 64 --cfg yolov5n6.yaml --weights '' --data coco.yaml --img 640 --epochs 300 --linear
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

#################################
### DO NOT ALTER THIS PORTION ###
#################################
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
#################################
### DO NOT ALTER THIS PORTION ###
#################################
###########################################
### Image Augmentation Parameters Start ###
###########################################
hsv_h: 0.0  # image HSV-Hue augmentation (fraction)
hsv_s: 0.0  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.0  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.0  # image translation (+/- fraction)
scale: 0.0  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.000  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.0  # image flip left-right (probability)
mosaic: 0.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
#########################################
### Image Augmentation Parameters End ###
#########################################
# copy_paste is for image segmentation leave this augmentation alone
copy_paste: 1.0  # segment copy-paste (probability)

## Training Parameters

- Find the explaination from train.py and then include more verbose explaination

`--img`: \
`--batch`: \
`--epochs`: \
`--data`: \
`--weights`: \
`--cache`: \
`--name`: 

In [ ]:
# give some explanation of the different parameters
!python train.py --img 640 --batch 16 --epochs 1 --data coco128.yaml --weights yolov5s.pt --cache --name aug-mosaic

## Tensorboard

- to insert description

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
# if you see a google 403 error please ensure that your browser allows 3rd party cookie for tensorboard to be viewable
%load_ext tensorboard
%tensorboard --logdir runs/train

## Visualising augmented images during training

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/aug-mosaic2/train_batch1.jpg', width=900)